In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import time

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
input_tensor   = tf.placeholder(shape=(None, 64, 64, 3), dtype="float32", name="input_tensor")
output_tensor  = tf.placeholder(shape=(None, 2), dtype="float32", name="output_tensor")
bn_tensor      = tf.placeholder(tf.bool,shape=None)
dropout_tensor = tf.placeholder(shape=(None), dtype="float32", name = "Dropout")

In [ ]:
layer_1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2, 2))(input_tensor)
layer_2 = tf.keras.layers.BatchNormalization()(layer_1, training = bn_tensor)
layer_3 = tf.keras.layers.ReLU()(layer_2)
layer_3 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_3)
layer_4 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (1,1), strides = (1,1))(layer_3)
layer_5 = tf.keras.layers.BatchNormalization()(layer_4, training = bn_tensor)
layer_6 = tf.keras.layers.ReLU()(layer_5)
layer_6 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_6)


layer_7  = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2, 2))(layer_6)
layer_8  = tf.keras.layers.BatchNormalization()(layer_7,training = bn_tensor)
layer_9  = tf.keras.layers.ReLU()(layer_8)
layer_9  = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_9)
layer_10 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (1,1), strides = (1,1))(layer_9)
layer_11 = tf.keras.layers.BatchNormalization()(layer_10, training = bn_tensor)
layer_12 = tf.keras.layers.ReLU()(layer_11)
layer_12 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_12)


layer_13 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (1, 1))(layer_12)
layer_14 = tf.keras.layers.BatchNormalization()(layer_13, training = bn_tensor)
layer_15 = tf.keras.layers.ReLU()(layer_14)
layer_15 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_15)
layer_16 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1))(layer_15)
layer_17 = tf.keras.layers.BatchNormalization()(layer_16, training = bn_tensor)
layer_18 = tf.keras.layers.ReLU()(layer_17)
layer_18 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_18)


layer_19 = tf.keras.layers.Conv2D(filters = 256, kernel_size = (3,3), strides = (2, 2))(layer_18)
layer_20 = tf.keras.layers.BatchNormalization()(layer_19, training = bn_tensor)
layer_21 = tf.keras.layers.ReLU()(layer_20)
layer_21 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_21)
layer_22 = tf.keras.layers.Conv2D(filters = 256, kernel_size = (1,1), strides = (1,1))(layer_21)
layer_23 = tf.keras.layers.BatchNormalization()(layer_22, training = bn_tensor)
layer_24 = tf.keras.layers.ReLU()(layer_23)
layer_24 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_24)

layer_25 = tf.keras.layers.AveragePooling2D(pool_size= 6, strides =1)(layer_24) 


In [ ]:
layer_26 = tf.keras.layers.Dense(units= 2)(layer_25)
layer_27 = tf.reshape(layer_26,shape = (-1, 2))

In [ ]:
print(layer_1)
print(layer_2)
print(layer_3)
print(layer_4)
print(layer_5)
print(layer_6)
print(layer_7)
print(layer_8)
print(layer_9)
print(layer_10)
print(layer_11)
print(layer_12)
print(layer_13)
print(layer_14)
print(layer_15)
print(layer_16)
print(layer_17)
print(layer_18)
print(layer_19)
print(layer_20)
print(layer_21)
print(layer_22)
print(layer_23)
print(layer_24)
print(layer_25)
print(layer_26)
print(layer_27)



In [ ]:
prediction = layer_27

In [ ]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=output_tensor, logits=prediction) # tf.reduce_mean(tf.squared_difference(output_tensor, prediction))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(output_tensor, 1)), "float32"))

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer()
    loss_optimization = optimizer.minimize(loss)

In [ ]:
images_list = []
for root, folder, files in os.walk("/home/grace/Projects/Dog_Vs_Cat/Train_data/"):
    for file in files :
        images_list.append(root + file)
images_list , test = images_list[0:20000] , images_list[20000:]
print("No of Train Images : ", len(images_list))
print("No of Test Images  : ", len(test))


In [ ]:
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)

In [ ]:
num_epochs = 100
batch = 10
total_images = len(images_list)
test_images = len(test)
for epoch in range(num_epochs):
    t1 = time.time()
    acc_list = []
    loss_list = []
    test_acc_list = []
    for i in range(0, total_images-batch, batch):
        image_batch = []
        label_batch = []
        for image in images_list[i:i+batch]:
            single_image = cv2.imread(image)/255.0 - 0.5
            single_image = cv2.resize(single_image, (64, 64))
            image_batch.append(single_image)
            label = int(image.split("/")[-1].split("_")[0])
            label_batch.append(np.eye(2)[label])
        label_batch = np.array(label_batch, dtype=np.float32)
        
        if epoch == 0:            
            acc, loss_val = sess.run([accuracy, loss], \
                                     feed_dict={input_tensor :image_batch, output_tensor : label_batch, \
                                               bn_tensor : False, dropout_tensor : 0.0 })
            acc_list.append(acc)
            loss_list.append(loss_val)
        else : 
            opt, acc, loss_val = sess.run([loss_optimization, accuracy, loss], \
                                          feed_dict={input_tensor :image_batch, output_tensor : label_batch, \
                                                    bn_tensor : True, dropout_tensor : 0.2 })
            acc_list.append(acc)
            loss_list.append(loss_val)
    for j in range(0, test_images-batch, batch):
        test_image_batch = []
        test_label_batch = []
        for test_image in test[j:j+batch]:
            single_test_image = cv2.imread(test_image)/255.0 - 0.5
            single_test_image = cv2.resize(single_test_image, (64, 64))
            test_image_batch.append(single_test_image)
            test_label = int(test_image.split("/")[-1].split("_")[0])
            test_label_batch.append(np.eye(2)[test_label])
        test_label_batch = np.array(test_label_batch, dtype=np.float32)
        
        test_acc = sess.run([accuracy], \
                            feed_dict={input_tensor :test_image_batch, output_tensor : test_label_batch, \
                                      bn_tensor : False, dropout_tensor : 0.0 })
        test_acc_list.append(test_acc)
        
    print("Epoch : " + str(epoch) + " Loss " +  str(round(np.average(loss_list), 3)) + \
          "  Train Accuracy : " + str(round(np.average(acc_list), 3)) + "  Test Accuracy : " + \
          str(round(np.average(test_acc_list), 3)))
    print("Time Takes For This Epoch is : ", time.time() - t1)
    print("******************************************************")

In [ ]:
saver = tf.train.Saver()
saver.save(sess,'./Dogs_vs_Cats_Model/TF_Model')


In [ ]:
frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,
    sess.graph_def,
    ["Reshape"])

with open('./Dogs_vs_Cats_Model/TF_FrozenModel.pb', 'wb') as f:
    f.write(frozen_graph_def.SerializeToString())